In [1]:
import pandas as pd
import os
import csv
import xlsxwriter

In [2]:
#Primer paso: unificar los archivos csv en una lista de listas
ruta_archivos = r"C:\Users\Usuario\Desktop\Programación\Proyecto Análisis de Datos Entrenos\Datos entrenos nuevos\Sentadilla Frontal"
nombre_archivo = "Datos Sentadilla Frontal.xlsx"

def unificar_archivos(ruta_archivos):
    lista_datos = []
    for archivo in os.listdir(ruta_archivos):
        if archivo.endswith('.csv'):
            ruta_archivo = os.path.join(ruta_archivos, archivo)
            with open(ruta_archivo, newline='') as archivo_csv:
                lector_csv = csv.reader(archivo_csv)
                for fila in lector_csv:
                    lista_datos.append(fila)
    return lista_datos

lista_datos = unificar_archivos(ruta_archivos)

In [3]:
#Segundo paso: convertir la lista de listas en un dataframe de pandas
def convertir_a_pandas(lista_datos):
    df = pd.DataFrame(lista_datos)
    return df

df = convertir_a_pandas(lista_datos)

In [4]:
#Paso 3: eliminar las filas que no nos interesan
df.drop(df.loc[df[0] == "Atleta"].index, inplace=True)
df.drop(df.loc[df[0] == "Marcado con *: Solo encoder con Bluetooth"].index, inplace=True)
df.drop(df.loc[df[0] == "Email"].index, inplace=True)

In [5]:
#Paso 4: Determinar que nombre quiero para mis columnas y elimnar esas filas
df.columns = df.iloc[0]
df.drop(df.loc[df["Ejercicio"] == "Ejercicio"].index, inplace=True)

In [6]:
#Paso 5: Eliminar las columnas que no me interesan
df.drop("Comentarios", axis=1, inplace=True)

In [7]:
#Reseteo el index
df.reset_index(drop=True, inplace=True)

In [8]:
#Paso 6: Igualar todas las filas en las que no haya datos en la columna ejercicio, fecha, etc.

def rellenar_datos_faltantes(columna):
    valor_del_momento = df.loc[0, columna]
    for indice, valor in df[columna].iteritems():
        if valor == "":
            df.loc[indice, columna] = valor_del_momento
        else:
            valor_del_momento = valor

for columna in df.columns:
    rellenar_datos_faltantes(columna)

In [9]:
#Cambio la fecha a formato date, elimino la hora
df["Fecha"] = pd.to_datetime(df["Fecha"], format="%d-%m-%Y %H:%M")
df["Fecha"] = df["Fecha"].dt.date


In [10]:
#Elimino las filas que no tienen datos en la columna serie y en la columna repeticion
df = df.dropna(subset=["nº Serie"])
df = df.dropna(subset=["nº Rep."])


In [11]:
#Reseteo el index
df.reset_index(drop=True, inplace=True)

In [12]:
#Transformo los datos de las columnas que van a contener datos numericos
columnas_no_numericas = ["Ejercicio", "Fecha", "Tipo*"]
for columna in df.columns:
    if columna not in columnas_no_numericas:
        df[columna] = df[columna].str.replace(",", ".")
        df[columna] = pd.to_numeric(df[columna])


In [13]:
#Convierto columna fecha a formato date
df["Fecha"] = pd.to_datetime(df["Fecha"], format="%Y-%m-%d")

In [14]:
#Convierto dataframe a excel
writer = pd.ExcelWriter(nombre_archivo, engine='xlsxwriter')
df.to_excel(writer, sheet_name='Sheet1')
writer.save()
